<a href="https://colab.research.google.com/github/MasahiroAraki/MLCourse/blob/master/12_apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第12章 パターンマイニング

## 頻出項目集合抽出

Aprioriアルゴリズムの実装

http://rasbt.github.io/mlxtend/api_subpackages/mlxtend.frequent_patterns/

In [1]:
# colabで実行
!pip install -U mlxtend
import pkg_resources, imp
imp.reload(pkg_resources)

Requirement already up-to-date: mlxtend in /usr/local/lib/python3.6/dist-packages (0.17.2)


<module 'pkg_resources' from '/usr/local/lib/python3.6/dist-packages/pkg_resources/__init__.py'>

In [0]:
import numpy as np
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth

例題12.1 のデータを読み込んで、pandasのDataFrameに変換。項目がアルファベット順にソートされていることに注意。

In [0]:
dataset = [['Milk', 'Bread', 'Butter'],
  ['Milk', 'Bread', 'Jam'],
  ['Milk', 'Magazine'],
  ['Bread', 'Butter'],
  ['Milk', 'Bread', 'Butter', 'Jam'],
  ['Magazine'],
  ['Milk', 'Bread', 'Jam', 'Magazine'],
  ['Jam']]

In [4]:
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,Bread,Butter,Jam,Magazine,Milk
0,True,True,False,False,True
1,True,False,True,False,True
2,False,False,False,True,True
3,True,True,False,False,False
4,True,True,True,False,True
5,False,False,False,True,False
6,True,False,True,True,True
7,False,False,True,False,False


Aprioriアルゴリズムで3事例以上出現する項目集合を抽出

In [5]:
freq = apriori(df, min_support= 3/len(df), use_colnames=True)
freq

,support,itemsets
0,0.625,(Bread)
1,0.375,(Butter)
2,0.500,(Jam)
3,0.375,(Magazine)
4,0.625,(Milk)
5,0.375,"(Butter, Bread)"
6,0.375,"(Jam, Bread)"
7,0.500,"(Milk, Bread)"
8,0.375,"(Jam, Milk)"
9,0.375,"(Jam, Bread, Milk)"


## 規則抽出

In [6]:
association_rules(freq, metric="confidence", min_threshold=0.7)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Butter),(Bread),0.375,0.625,0.375,1.00,1.60,0.140625,inf
1,(Jam),(Bread),0.500,0.625,0.375,0.75,1.20,0.062500,1.500
2,(Milk),(Bread),0.625,0.625,0.500,0.80,1.28,0.109375,1.875
3,(Bread),(Milk),0.625,0.625,0.500,0.80,1.28,0.109375,1.875
4,(Jam),(Milk),0.500,0.625,0.375,0.75,1.20,0.062500,1.500
5,"(Jam, Bread)",(Milk),0.375,0.625,0.375,1.00,1.60,0.140625,inf
6,"(Jam, Milk)",(Bread),0.375,0.625,0.375,1.00,1.60,0.140625,inf
7,"(Milk, Bread)",(Jam),0.500,0.500,0.375,0.75,1.50,0.125000,2.000
8,(Jam),"(Milk, Bread)",0.500,0.500,0.375,0.75,1.50,0.125000,2.000


supermarket.arffでFPGrowth（Aprioriの高速化版）を試してみます。データをダウンロードし、scipyのloadarffでWekaのarff形式のデータを読み込みます。その際、読み込みエラーを回避するために文字列置換を行い、最終的にpandasのDataFrameで値をTrue/Falseとし、最終列のtotalは除きます。

In [7]:
!wget https://raw.githubusercontent.com/fracpete/wekamooc/master/dataminingwithweka/data/supermarket.arff

--2020-06-13 07:14:18--  https://raw.githubusercontent.com/fracpete/wekamooc/master/dataminingwithweka/data/supermarket.arff
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2025871 (1.9M) [text/plain]
Saving to: ‘supermarket.arff.1’

supermarket.arff.1  100%[===================>]   1.93M  --.-KB/s    in 0.1s    

2020-06-13 07:14:18 (15.9 MB/s) - ‘supermarket.arff.1’ saved [2025871/2025871]



In [0]:
!sed -e "s/{ t}/{t}/" supermarket.arff > supermarket2.arff

In [9]:
from scipy.io import arff
data, meta = arff.loadarff('supermarket2.arff')
df = pd.DataFrame(data)
df2 = df.replace({b'?':False, b't':True})
df2 = df2.drop('total', axis=1)
df2

,department1,department2,department3,department4,department5,department6,department7,department8,department9,grocery misc,department11,baby needs,bread and cake,baking needs,coupons,juice-sat-cord-ms,tea,biscuits,canned fish-meat,canned fruit,canned vegetables,breakfast food,cigs-tobacco pkts,cigarette cartons,cleaners-polishers,coffee,sauces-gravy-pkle,confectionary,puddings-deserts,dishcloths-scour,deod-disinfectant,frozen foods,razor blades,fuels-garden aids,spices,jams-spreads,insecticides,pet foods,laundry needs,party snack foods,...,department177,department178,department179,casks white wine,casks red wine,750ml white nz,750ml red nz,750ml white imp,750ml red imp,sparkling nz,sparkling imp,brew kits/accesry,department189,port and sherry,ctrled label wine,department192,department193,department194,department195,department196,department197,department198,department199,non host support,department201,department202,department203,department204,department205,department206,department207,department208,department209,department210,department211,department212,department213,department214,department215,department216
0,False,False,False,False,False,False,False,False,False,False,False,True,True,True,False,True,False,True,False,False,True,False,False,False,True,True,True,True,False,True,False,True,True,False,False,False,False,False,False,True,...,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,False,False,False,False,False,True,False,False,False,True,True,False,False,False,False,False,True,True,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,True,False,True,False,True,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,False,False,True,True,False,True,False,True,False,False,True,True,False,False,True,False,False,False,False,False,False,True,False,False,False,True,False,True,False,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,True,True,False,False,False,False,False,False,False,True,True,True,False,False,False,False,True,False,False,False,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4622,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,True,False,True,False,True,True,False,False,False,True,False,True,False,True,Fa

FPGrowthで頻出項目集合を求め、association_rulesで規則を抽出します。

In [0]:
freq2 = fpgrowth(df2, min_support=0.3, use_colnames=True)

In [11]:
association_rules(freq2, metric="lift", min_threshold=1.2)



,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(vegetables, bread and cake)",(fruit),0.496650,0.640156,0.387076,0.779373,1.217475,0.069143,1.631011
1,"(bread and cake, fruit)",(vegetables),0.502485,0.639939,0.387076,0.770323,1.203743,0.065516,1.567679
2,(vegetables),"(bread and cake, fruit)",0.639939,0.502485,0.387076,0.604863,1.203743,0.065516,1.259095
3,(fruit),"(vegetables, bread and cake)",0.640156,0.496650,0.387076,0.604659,1.217475,0.069143,1.273204
4,"(vegetables, milk-cream)",(fruit),0.437649,0.640156,0.339529,0.775802,1.211897,0.059366,1.605033
5,"(milk-cream, fruit)",(vegetables),0.440458,0.639939,0.339529,0.770854,1.204573,0.057662,1.571313
6,(vegetables),"(milk-cream, fruit)",0.639939,0.440458,0.339529,0.530564,1.204573,0.057662,1.191945
7,(fruit),"(vegetables, milk-cream)",0.640156,0.437649,0.339529,0.530385,1.211897,0.059366,1.197473
8,"(baking needs, fruit)",(vegetables),0.410633,0.639939,0.321807,0.783684,1.224622,0.059026,1.664513
9,(vegetables),"(baking needs, fruit)",0.639939,0.410633,0.321807,0.502871,1.224622,0.059026,1.185540


## 行列分解

小規模な映画評価データを使って、行列分解を行います。



ライブラリの読み込み

In [0]:
import numpy as np
from sklearn.decomposition import NMF

<a href="http://www.quuxlabs.com/blog/2010/09/matrix-factorization-a-simple-tutorial-and-implementation-in-python/">参考ページ</a>のデータ例を使用します。縦がユーザ、横が映画、数値が1-5の5段階評価で、0は評価なしです。

In [0]:
X = np.array([
    [5,3,0,1],
    [4,0,0,1],
    [1,1,0,5],
    [1,0,0,4],
    [0,1,5,4]
])

ここでは、非負値行列分解NMFを使います。NMFはXを非負の行列WとHに分解します。引数n_componentsは潜在変数の次元数です。

In [0]:
model = NMF(n_components = 2)
W = model.fit_transform(X)
H = model.components_

空所の値を予測してみます。似たようなユーザを参考に、埋められた値の妥当性を評価してみてください。

In [15]:
np.set_printoptions(suppress=True)
np.dot(W,H)

array([[5.2558264 , 1.99313836, 0.        , 1.45512772],
       [3.50429478, 1.32891458, 0.        , 0.9701988 ],
       [1.31294288, 0.94415991, 1.94956896, 3.94609389],
       [0.98129195, 0.72179987, 1.52759811, 3.0788454 ],
       [0.        , 0.65008935, 2.84003662, 5.21894555]])

人を表す2次元ベクトルを表示します。同じような評価をする人が似たベクトルになっていることを確認してください。

In [16]:
W

array([[0.        , 2.37453733],
       [0.        , 1.58321035],
       [1.35650216, 0.59317634],
       [1.06289656, 0.44333929],
       [1.97608595, 0.        ]])

映画を表す2次元ベクトルを表示します。同じような評価をされる映画がないので、それぞれ異なったベクトルになっていることを確認してください。

In [17]:
H.T

array([[0.        , 2.21341073],
       [0.32897828, 0.83937967],
       [1.43720298, 0.        ],
       [2.6410519 , 0.61280474]])

## 課題1

上記の規則抽出のコード例で求めた規則よりも、Lift値の高い規則を探すにはどうすればよいでしょうか。

## 課題2

教科書p.226の演習問題12-3の手順に従い、MovieLensデータセットに対してscikit-surpriseを用いてNMFを行ってください。余力があれば、ハイパーパラメータを変更して、性能に与える影響を観察してください。